<a href="https://colab.research.google.com/github/shaja-asm/EMLEC/blob/dev/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projection, Layernorm, Dropout

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/SkillSurf/introduction_genAI/blob/main/notebooks/S1/Session_1_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

<br />
<br />

In [76]:
import torch
import torch.nn as nn
from torch.nn import functional as F

### Hyperparameters

In [77]:
'''Hyperparameters for smaller model'''

# B = 32 # B: how many independent sequences will we process in parallel?
# T = 8  # T: what is the maximum context length for predictions?
# C = 32 # C: numer of different features analysed (also D = dims)
# H = 4  # H: number of attention heads
# L = 4  # L: Number of layers
# learning_rate = 1e-3

'''Final Hyperparameters'''

B = 64 # B: how many independent sequences will we process in parallel?
T = 256  # T: what is the maximum context length for predictions?
H = 6
C = 64*H
L = 6
learning_rate = 1e-4

# Common Hyperparameters
max_iters = 5000
eval_interval = 500
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
dropout = 0.2
torch.manual_seed(1337)

### Data

In [78]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-04 07:28:42--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.7’

input.txt.7         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-08-04 07:28:42 (27.4 MB/s) - ‘input.txt.7’ saved [1115394/1115394]



In [79]:
import re

# Read the input text from a file
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Split text into words based on spaces, newlines, double newlines, and commas
words = re.split(r'[ ,\n]+', text)

# Remove punctuation from each word and filter out empty strings
words = [re.sub(r'[^\w\s]', '', word) for word in words if word]

# Create a set of unique words
unique_words = sorted(set(words))
vocab_size = len(unique_words)

# Create a mapping from words to integers
stoi = {word: i for i, word in enumerate(unique_words)}
itos = {i: word for i, word in enumerate(unique_words)}

# Encoder: take a string, output a list of integers
encode = lambda s: [stoi[word] for word in re.split(r'[ ,\n]+', s) if word]

# Decoder: take a list of integers, output a string
decode = lambda l: ' '.join([itos[i] for i in l])

# Output the vocabulary size and the vocabulary
unique_words_str = ', '.join(unique_words)
print(f'vocab_size: {vocab_size}')
print(f'vocabulary: {unique_words_str}')


vocab_size: 14668
vocabulary: , 3, A, ABHORSON, ABRAHAM, ADRIAN, AEacides, AEdile, AEdiles, AEneas, AEsop, ALL, ALONSO, ANGELO, ANNE, ANOTHER, ANTIGONUS, ANTONIO, ARCHBISHOP, ARCHIDAMUS, ARIEL, AUFIDIUS, AUMERLE, AUTOLYCUS, Abase, Abate, Abated, Abbot, Abels, Abhorred, Abhorson, Abides, Able, About, Above, Abraham, Abrahams, Absolute, Accept, Accomplishd, According, Accords, Account, Accountant, Accursed, Accuse, Achieve, Acquaint, Action, Adam, Adams, Add, Added, Adding, Address, Adieu, Adjudged, Admit, Adonis, Adoptedly, Adopts, Adrian, Adriatic, Advance, Advantaging, Adversitys, Advertising, Advocates, Affection, Affections, Affliction, Affrighted, Affrights, Affront, Afore, Afresh, Afric, African, After, Again, Against, Agamemnons, Age, Aged, Agenor, Agreed, Agrippa, Ah, Aimd, Aiming, Airy, Ajax, Alack, Alas, Albans, Albeit, Albion, Albions, Alce, Alcides, Alexander, Alike, Alive, All, AllSeer, AllSouls, Alla, Allaying, Allhallond, Allot, Allow, Allowing, Alls, Almost, Alone, Alps,

### Head, MHSA

In [80]:
class Head(nn.Module):
    """ One head of self attention"""

    def __init__(self, Ci, Co):
        super().__init__()
        self.key   = nn.Linear(Ci, Co, bias=False)
        self.query = nn.Linear(Ci, Co, bias=False)
        self.value = nn.Linear(Ci, Co, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(T, T)))

    def forward(self, x):
        B, T, Ci  = x.shape
        '''
        B  - batch               # of independant vectors processed
        T  - time/block/context  # of tokens in a context
        Ci - channals/dims input # of features in input
        '''

        k = self.key(x)   # (B,T,Co)
        q = self.query(x) # (B,T,Co)

        # compute attention scores / affinities
        wei = q @ k.transpose(-2,-1)                                 # (B,T,Co) @ (B,Co,T) -> (B,T,T)
        wei /= C**0.5                                                # (B,T,T) scaling, bring variance to 1, to prevent softmax clipping
        wei  = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))   # (B,T,T) Replace upper triangular of wei with -inf
        wei  = F.softmax(wei, dim=-1)                                # (B,T,T) -inf -> 0, rest normalized to 1

        v = self.value(x)  # (B,T,Co)
        out = wei @ v      # (B,T,T) @ (B,T,Co) = (B,T,Co)

        return out


class MultiHeadAttention(nn.Module):

    def __init__(self, Ci, H, head_size):
        super().__init__()
        # 4 heads of 8-dimensional self-attention, for n_embed=32, like a group convolution
        self.heads = nn.ModuleList([Head(Ci=Ci, Co=head_size) for _ in range(H)])

    def forward(self, x):
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        return x

### Transformer Block

In [81]:
class Block(nn.Module):
    ''' Transformer block: communication followed by computation '''

    def __init__(self, C, H): # C: embedding dimension, H: number of heads
        super().__init__()
        self.ln1 = nn.LayerNorm(C)   # Layernorm along channels (batch & time are batch dims): y = beta + gamma * [x-E(x)]/sqrt(V(x) + ep)
        self.sa = MultiHeadAttention(Ci=C, H=H, head_size=C//H)
        self.ln2 = nn.LayerNorm(C)
        self.ffwd = nn.Sequential(         # Feedforward network, so the tokens can "think about" what they found in attention.
            nn.Linear(C, C*4),
            nn.GELU(),
            nn.Linear(C*4, C),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        # Residual connections around MSA & FF, to help training
        # Note: input without layernorm is added to output

        x_skip = x

        x = self.ln1(x)
        x = self.sa(x)   # (B,T,C), Multi head self attention
        x = x + x_skip

        x = self.ln2(x)
        x = self.ffwd(x) # (B,T,C), Per token level. B,T act as batch dimensions
        x = x + x_skip

        return x


### Model

In [82]:
class BigramLanguageModel(nn.Module):

    def __init__(self, B,T,C,H,L):
        super().__init__()
        self.B, self.T, self.C, self.H, self.L = B,T,C,H,L
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, C) # for every possible token, weights for next token
        self.position_embedding_table = nn.Embedding(T, C)

        self.blocks  = nn.Sequential(*[Block(C, H) for _ in range(L)])
        self.ln_final = nn.LayerNorm(C)
        self.lm_head = nn.Linear(C, vocab_size)

    def forward(self, idx, targets=None):

        tok_emb = self.token_embedding_table(idx)                                    # (B,T,C=n_embed)
        pos_emb = self.position_embedding_table(torch.arange(self.T, device=device)) # (T,C): [0,1,2..T-1]

        x = tok_emb + pos_emb     # (B,T,C)
        x = self.blocks(x)
        x = self.ln_final(x)      # Layernorm applied before last
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):                        # idx is (B, T) array of indices in the current context
            idx_cond = idx[:, -self.T:]                        # crop the last block_size tokens for input
            logits, loss = self(idx_cond)                      # get the predictions
            logits = logits[:, -1, :]                          # (B,T,C) -> (B, C)
            probs = F.softmax(logits, dim=-1)                  # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # sample from the distribution acc to prob (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)            # New idx is concat (B, T+1)
        return idx

model = BigramLanguageModel(B,T,C,H,L)
m = model.to(device)

### Training

In [83]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0:   # every once in a while evaluate the loss on train and val sets
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')     # sample a batch of data

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 9.7647, val loss 9.7635
step 500: train loss 2.4430, val loss 2.4634
step 1000: train loss 2.2119, val loss 2.2506
step 1500: train loss 1.9475, val loss 2.0481
step 2000: train loss 1.7326, val loss 1.8889
step 2500: train loss 1.5990, val loss 1.7852
step 3000: train loss 1.5040, val loss 1.7095
step 3500: train loss 1.4291, val loss 1.6559
step 4000: train loss 1.3808, val loss 1.6182
step 4500: train loss 1.3412, val loss 1.5919


### Inference

In [84]:
context = torch.ones((1, T), dtype=torch.long, device=device)  # start with '\n\n\n\n' as seed
out_ints = m.generate(context, max_new_tokens=2000)[0].tolist() # output list of ints
print(decode(out_ints))

3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 ARCHBISHOP Abhorson Acquaint Add AEsop  ANNE Accountant AEacides 3 Adonis Achieve Accountant Accomplishd Adjudged Accountant 3 Adding Accursed Acquaint Adams Acquaint Adonis Achieve 3 Accomplishd 3 Account Accomplishd According Adjudged Adonis Achieve AEacides 3 According Accountant 3 Acquaint Added Adonis Accountant Adjudged Accountant 3 Adding Added Accountant AEdiles   ANGELO About ANTIGONUS AUFIDIUS Abhorson AUTOLYCUS ANTONIO Abase ANTONIO Abase AEsop  Able Achieve Accountant 3 A